# What is Gradio?

Gradio is an open-source Python library that lets you create **web-based GUIs** from your machine learning (ML) models in just a few lines of code. These GUIs allow your models to be **used and tested by non-ML people** without any specialized hardware or software, directly from the browser!

![link text](https://i.imgur.com/MCfZmjd.png)


# How do I use Gradio?

### 0. Installation

`gradio` is a Python package, so it can be installed with `pip`.

In [1]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.8 MB/s eta 0:

### 1. Basic Usage: "Hello World"

At it's core, `gradio` allows you to wrap UI elements around Python functions. There are a variety of UI elements including:

* `Image` (drag-and-drop interface)
* `Text` (write freeform text)
* `Microphone` (record audio)
* `Slider` (select a numerical value)

and [many, many more](https://gradio.app/docs)! These are just the input interfaces. Most of these have corresponding output interfaces as well. With gradio, you simply combine 3 ingredients: **a Python function**, an **input component**, and an **output component**, and you're ready to go!

Let's start with a basic example:

In [2]:
import gradio as gr
import time

def replace(text):
    return text.replace('World', 'Databricks')

gr.Interface(fn=replace,
             inputs='textbox',
             outputs='textbox').launch(share=True);


""" Equivalent Code:
gr.Interface(fn=replace,
             inputs=gr.inputs.Textbox(),
             outputs=gr.outputs.Textbox()).launch();
""";

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2b5101bd85a6a6e93a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 2. MNIST Example

Now that we've gotten the basic idea, let's try with a more interesting example, a machine learning model:

In [3]:
import tensorflow as tf
import numpy as np
from urllib.request import urlretrieve
import gradio as gr

urlretrieve("https://gr-models.s3-us-west-2.amazonaws.com/mnist-model.h5", "mnist-model.h5")
model = tf.keras.models.load_model("mnist-model.h5")

def recognize_digit(image):
    image = image.reshape(1, -1)  # add a batch dimension
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

output_component = gr.outputs.Label(num_top_classes=3)

gr.Interface(fn=recognize_digit,
             inputs="sketchpad",
             outputs=output_component).launch(share=True);

<ipython-input-3-24ed89e3fe1b>:14: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output_component = gr.outputs.Label(num_top_classes=3)
<ipython-input-3-24ed89e3fe1b>:14: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  output_component = gr.outputs.Label(num_top_classes=3)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c60b0ba039ea7138b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 2b. MNIST (Live Predictions)

In [4]:
gr.Interface(fn=recognize_digit,
             inputs="sketchpad",
             outputs=output_component, live=True).launch();

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### 3. Manipulating Images

In [5]:
import gradio as gr
import tensorflow as tf
import numpy as np
import requests

inception_net = tf.keras.applications.InceptionV3() # load the model

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify_image(inp):
  inp = np.expand_dims(inp, 0)
  inp = tf.keras.applications.inception_v3.preprocess_input(inp)
  prediction = inception_net.predict(inp).flatten()
  return {labels[i]: float(prediction[i]) for i in range(1000)}

image = gr.inputs.Image(shape=(299, 299))
label = gr.outputs.Label(num_top_classes=3)

gr.Interface(fn=classify_image, inputs=image, outputs=label).launch(share=True);

96112376/96112376 [==============================] - 2s 0us/step


<ipython-input-5-b7c7ebbfb36d>:18: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(299, 299))
<ipython-input-5-b7c7ebbfb36d>:18: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(299, 299))
<ipython-input-5-b7c7ebbfb36d>:19: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  label = gr.outputs.Label(num_top_classes=3)
<ipython-input-5-b7c7ebbfb36d>:19: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  label = gr.outputs.Label(num_top_classes=3)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a2324d77408d5a1f1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 3b. Adding Interpretation (*NEW*)

In [ ]:
gr.Interface(fn=classify_image, inputs=image, outputs=label, interpretation="default").launch();

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://37079.gradio.app
Interface loading below...


### How does sharing and flagging work?

![link text](https://gradio.app/static/home/img/sharing.svg)

### 4. Multiple Inputs and Outputs, with Timeseries

In [ ]:
import gradio as gr
import pandas as pd
import random


def storm_predictor(data, categories, sensitivity):
    activity_range = random.randint(0, 100)
    return {"Storm": activity_range / 100., "No storm": 1 - activity_range / 100.}

iface = gr.Interface(storm_predictor,
                     [
                         gr.inputs.Timeseries(
                             x="time",
                             y=["wind", "temperature", "precipitation"]
                         ),
                         gr.inputs.CheckboxGroup(["wind", "temperature", "precipitation"], default=[
                             "wind", "temperature", "precipitation"]),
                         gr.inputs.Slider(1, 3)
                     ],
                     gr.outputs.Label(label="Risk of storm"),
                     )

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://14652.gradio.app
Interface loading below...


### 5. Comparing Multiple Models in Parallel and Adding Metadata

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from urllib.request import urlretrieve

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

# Download sample images
urlretrieve("https://www.sciencemag.org/sites/default/files/styles/article_main_large/public/cc_BE6RJF_16x9.jpg?itok=nP17Fm9H","monkey.jpg")
urlretrieve("https://www.discoverboating.com/sites/default/files/inline-images/buying-a-sailboat-checklist.jpg","sailboat.jpg")
urlretrieve("https://external-preview.redd.it/lG5mI_9Co1obw2TiY0e-oChlXfEQY3tsRaIjpYjERqs.jpg?auto=webp&s=ea81982f44b83efbb803c8cff8953ee547624f70","bicycle.jpg")
urlretrieve("https://www.chamaripashoes.com/blog/wp-content/uploads/2018/09/Robert-Downey-Jr..jpg","rdj.jpg")

mobile_net = tf.keras.applications.MobileNetV2()
inception_net = tf.keras.applications.InceptionV3()

def classify_image_with_mobile_net(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((224, 224))
  arr = np.array(im).reshape((-1, 224, 224, 3))
  arr = tf.keras.applications.mobilenet.preprocess_input(arr)
  prediction = mobile_net.predict(arr).flatten()
  return {labels[i]: float(prediction[i]) for i in range(1000)}


def classify_image_with_inception_net(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((299, 299))
  arr = np.array(im).reshape((-1, 299, 299, 3))
  arr = tf.keras.applications.inception_v3.preprocess_input(arr)
  prediction = inception_net.predict(arr).flatten()
  return {labels[i]: float(prediction[i]) for i in range(1000)}

imagein = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=3)
sample_images = [["monkey.jpg"],
                 ["rdj.jpg"],
                 ["sailboat.jpg"],
                 ["bicycle.jpg"]]

gr.Interface(
    [classify_image_with_mobile_net, classify_image_with_inception_net],
    imagein,
    label,
    title="MobileNet vs. InceptionNet",
    description="Compare 2 state-of-the-art machine learning models",
    examples=sample_images).launch();

14540800/14536120 [==============================] - 0s 0us/step
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://43217.gradio.app
Interface loading below...


# Next Steps

To get a sense of what kind of interfaces you can create, check out our model hub: www.gradio.app/hub

![link text](https://i.imgur.com/V4BbZhg.png)

Share your interfaces

In [ ]:
import gradio as gr
import pandas as pd
import time

def predict(file_obj,Question):
    time.sleep(200)
    df = pd.read_csv(file_obj.name,dtype=str)
    return df


io = gr.Interface(predict, ["file",gr.inputs.Textbox(placeholder="Enter Question here...")], "dataframe")
io.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://46051.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7862/',
 'https://46051.gradio.app')